# AI 법률 서비스

형량 예측

## 1. DataSet Load

In [15]:
from datasets import load_dataset
# legal judgment prediction
dataset = load_dataset("lawcompany/KLAID", 'ljp')

dataset['train']['fact'][:10]


Generating train split: 100%|██████████| 161192/161192 [00:00<00:00, 198490.80 examples/s]


['피고인은 2018. 8. 9. 23:33경 술을 마신 상태로 경산시 사동에 있는 상호 불상의 식당에서부터 같은 동에 있는 부영5차 앞 삼거리까지 B 스타렉스 승용차를 운전한 다음 승용차 안에서 잠을 자던 중, 차량 운전자가 시동을 걸어 놓고 잠을 자고 있다는 112 신고를 받고 현장에 출동한 경산경찰서 C파출소 소속 경위 D으로부터 피고인의 입에서 술 냄새가 나고 보행이 비틀거리는 등 술에 취한 상태에서 운전하였다고 인정할 만한 상당한 이유가 있어 약 10분 동안 총 3회에 걸쳐 음주측정기에 입김을 불어 넣는 방법으로 음주측정에 응할 것을 요구받고도 정당한 사유 없이 이에 응하지 아니하였다.',
 '피고인은 2016. 3. 19. 10:16경 경북 칠곡군 왜관읍 왜관대교 앞 도로에서부터 같은 군 북삼읍 칠곡대로 78 효성실버요양원 앞 도로에 이르기까지 약 8km 구간에서 자동차운전면허를 받지 아니하고 B 봉고Ⅲ 화물차를 운전하였다.',
 '피고인은 2016. 10. 10 16:55경 평택시 오성면 복합화력발전소 앞 도로에서부터 평택시 안중읍 석정삼거리 앞 도로까지 약 12km 구간에서 자동차운전면허를 받지 아니하고 B 포터초장축더블캡 화물차를 운전하였다.',
 '피고인 A은 노동일에 종사 중이다. 피고인은 2017. 2. 2. 20:00경 부산 해운대구 B, C식당 앞 노상에서 술을 마시고 D의 포터차량을 타려다가 주변에 있던 E 일행이 음주운전을 하려 한다고 말을 건 일로 서로 시비가 되었다. 피고인은 피해자 E과 시비를 벌이다. 피해자를 밀치고, 인근 주차장에서 들고 온 약 1.5m 길이의 쇠막대기로 E의 머리와 어깨 부위를 4~5회 가량 내리쳐 피해자를 폭행하였다.',
 '피고인은 2015. 7. 2. 06:35경 부산 부산진구 B에 있는 C슈퍼 앞길에서 피해자 D(66세) 운행의 택시에서 피고인이 분실한 신용카드를 피해자가 찾아주지 않고 그냥 가려고 한다는 이유로 피해자에게 “이 개쌔끼야, 씹할 놈아, 카드가 와 없노”라고 욕설을 하면서 택시 앞을 

## 2. RAG 용으로 문서 변환

In [2]:
from operator import itemgetter

from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

In [3]:
# Do not use this
# Embeddings are not supported
from langchain_openai import OpenAIEmbeddings
inference_server_url="https://8530-14-35-173-251.ngrok-free.app/v1/embeddings"
embeddings = OpenAIEmbeddings(model="./for_vllm", 
                              openai_api_key="EMPTY",
                              openai_api_base=inference_server_url,)

In [16]:
from langchain_core.documents import Document
doc=[]
for doc_ind in range(10): # [:10]
    doc_content = dataset['train']['fact'][doc_ind]
    if type(doc_content) != str:
        continue
    doc.append(Document(page_content=doc_content, metadata={"source": dataset['train']['laws_service'][doc_ind]}))

In [17]:
print(doc[1])

page_content='피고인은 2016. 3. 19. 10:16경 경북 칠곡군 왜관읍 왜관대교 앞 도로에서부터 같은 군 북삼읍 칠곡대로 78 효성실버요양원 앞 도로에 이르기까지 약 8km 구간에서 자동차운전면허를 받지 아니하고 B 봉고Ⅲ 화물차를 운전하였다.' metadata={'source': '도로교통법 제152조 제1호,도로교통법 제43조'}


In [18]:
embeddings = OllamaEmbeddings(model="llama3.1")

In [19]:
vectorstore = FAISS.from_documents(
    doc,
    embedding=embeddings
)

In [ ]:
vectorstore.save_local("faiss_index2")

In [ ]:
vectorstore.save_local("faiss_index2")

new_db = FAISS.load_local("faiss_index2", embeddings)

#docs = new_db.similarity_search(query)

In [9]:
#ds['train']['전문'][:10]

['【원고, 상고인】\n중앙재목주식회사 우대표자 취체역 김박일\n【피고, 피상고인】\n박삼준\n【주 문】\n원판결을 파훼함.\n본건을 서울지방심리원에 차려함.\n【이 유】\n상고이유 제1점은 원판결은 피고가 본건 가대를 최초에는 즉 1945년 10월 9일에는 김박일 개인으로부터 매수하였던 바 피고가 동월10일 등기부를 열람한 결과 본건 가대가 김박일의 개인의 소유가 아니고 원고회사 소유인 것을 발견하고 피고는 차 가옥을 동월 12일 원고회사로부터 다시 매수하였다고 즉 원고회사 대표취체역 김박일로부터 매수하였다고 인정하고 원고의 청구를 기각하였으나 원판결은 본건 매매를 부당하게 인정한 위법이 유한지라. 설사 원판결이 확정시킨 사실을 그대로 사실이라 할지라도 피고가 최초에 김박일 개인으로부터 매수하였다가 등기부를 열람한 결과 회사소유인 것을 발견하고 즉시 이것을 원고회사 대표취체역 김박일로부터 매수한 것 같이 작위한 것은 설사 김박일은 그와 같은조처를 취하였다 할지라도 그것은 형식상 합법화를 가장한데 불과한 것이며 실지에 있어서는 원고가 피고에게 이것을 매도하였다고 볼 수는 없는 사실이다. 즉 피고가 최초에 매수할 때에는 김박일 개인과의 매매이며 김박일 자신도 또한 개인소유로서 매도한 것으로 피고가 이러한 정세하에서 매수한 것은 김박일이 원고회사 소유가옥을 함부로 매각처분하는 것을 충분히 지한 것이며 또한 김박일이 회사재산을 이렇게 단단이 처분하는 줄로 알면서 다시 회사로부터 매수하는 것 같은 형식을 취하였으나 결국은 피고는 김박일이가 최초에 회사재산을 부정처분하려는 것을 발견하였음에도 불구하고 역시 그 대표권 남용을 알면서 다시금 회사로부터 매수하는 형식으로 김박일과 매매를 체결한것은 피고는 회사에 대하여는 악의의 제3자라고 볼것이므로 피고가 설사 김박일을 원고회사의 대표자로서의 자격으로 하고 매수하였다 하기로 이것으로써 원고에게 대항할 수 없으리라고 생각되는 바라. 하물며 본건 매매대금이라는 것은 원고회사는 이것을 취득한 일이 없고 그것을 모두 김박일이가 개인

In [20]:
db_retriever = vectorstore.as_retriever()
db_retriever.invoke("교통사고로 급발진 사고")

[Document(metadata={'source': '형법 제260조 제1항'}, page_content='피고인 A은 노동일에 종사 중이다. 피고인은 2017. 2. 2. 20:00경 부산 해운대구 B, C식당 앞 노상에서 술을 마시고 D의 포터차량을 타려다가 주변에 있던 E 일행이 음주운전을 하려 한다고 말을 건 일로 서로 시비가 되었다. 피고인은 피해자 E과 시비를 벌이다. 피해자를 밀치고, 인근 주차장에서 들고 온 약 1.5m 길이의 쇠막대기로 E의 머리와 어깨 부위를 4~5회 가량 내리쳐 피해자를 폭행하였다.'),
 Document(metadata={'source': '형법 제314조 제1항'}, page_content='피고인은 2015. 7. 2. 06:35경 부산 부산진구 B에 있는 C슈퍼 앞길에서 피해자 D(66세) 운행의 택시에서 피고인이 분실한 신용카드를 피해자가 찾아주지 않고 그냥 가려고 한다는 이유로 피해자에게 “이 개쌔끼야, 씹할 놈아, 카드가 와 없노”라고 욕설을 하면서 택시 앞을 가로막고 백미러를 잡아당기는 등 약 10분 동안 소란을 피워 피해자의 택시 영업업무를 방해하였다.'),
 Document(metadata={'source': '형법 제245조'}, page_content='피고인은 2014. 7. 9. 13:50경 광주 서구 무진대로 957 씨엘병원 앞 버스정류장 부근에서 피해자 C(여, 22세), 이름을 알 수 없는 여성 등이 버스를 기다리고 있는 것을 발견하고 D 소렌토 승용차를 위 피해자의 바로 앞에 주차한 다음 위 피해자를 보면서 승용차 운전석에 앉아 자신의 성기를 꺼내어 만지고 자위행위를 하는 등 공연히 음란한 행위를 하였다.'),
 Document(metadata={'source': '형법 제257조 제1항'}, page_content='피고인은 2019. 2. 25. 12:10경 전주시 덕진구 B에 있는 C에서 피해자 D(39세)과 함께 일용직 일을 마친 후 점심을 먹으려 하던 중, 피해자로부터 점심값을 계산

In [25]:
results = db_retriever.invoke("교통사고로 급발진 사고")
print(results[0].page_content)
print(results[1].metadata['source'])

피고인 A은 노동일에 종사 중이다. 피고인은 2017. 2. 2. 20:00경 부산 해운대구 B, C식당 앞 노상에서 술을 마시고 D의 포터차량을 타려다가 주변에 있던 E 일행이 음주운전을 하려 한다고 말을 건 일로 서로 시비가 되었다. 피고인은 피해자 E과 시비를 벌이다. 피해자를 밀치고, 인근 주차장에서 들고 온 약 1.5m 길이의 쇠막대기로 E의 머리와 어깨 부위를 4~5회 가량 내리쳐 피해자를 폭행하였다.
형법 제314조 제1항


In [21]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain_openai import ChatOpenAI

In [38]:
inference_server_url = "https://8530-14-35-173-251.ngrok-free.app/v1"

llm = ChatOpenAI(
    model="for_vllm",
    openai_api_key="EMPTY",
    openai_api_base=inference_server_url,
    max_tokens=500,
    temperature=0,
)

In [27]:
results = vectorstore.similarity_search_with_score("교통사고 급발진 사고")
print(results)

[(Document(metadata={'source': '형법 제260조 제1항'}, page_content='피고인 A은 노동일에 종사 중이다. 피고인은 2017. 2. 2. 20:00경 부산 해운대구 B, C식당 앞 노상에서 술을 마시고 D의 포터차량을 타려다가 주변에 있던 E 일행이 음주운전을 하려 한다고 말을 건 일로 서로 시비가 되었다. 피고인은 피해자 E과 시비를 벌이다. 피해자를 밀치고, 인근 주차장에서 들고 온 약 1.5m 길이의 쇠막대기로 E의 머리와 어깨 부위를 4~5회 가량 내리쳐 피해자를 폭행하였다.'), 18550.465), (Document(metadata={'source': '형법 제314조 제1항'}, page_content='피고인은 2015. 7. 2. 06:35경 부산 부산진구 B에 있는 C슈퍼 앞길에서 피해자 D(66세) 운행의 택시에서 피고인이 분실한 신용카드를 피해자가 찾아주지 않고 그냥 가려고 한다는 이유로 피해자에게 “이 개쌔끼야, 씹할 놈아, 카드가 와 없노”라고 욕설을 하면서 택시 앞을 가로막고 백미러를 잡아당기는 등 약 10분 동안 소란을 피워 피해자의 택시 영업업무를 방해하였다.'), 19745.37), (Document(metadata={'source': '특정경제범죄가중처벌등에관한법률 제3조 제1항 제2호,형법 제347조 제1항'}, page_content='공소장 기재 범죄사실 중 일부를 피고인의 방어권과 공소사실의 동일성을 해하지 않는 범위에서 수정하였다. 피고인은 피해자 C과 4촌 인척관계에 있는 사람이다. 피고인은 2014. 6. 초순경 안양시 만안구 D 건물 8층 사무실에서, 피해자 C에게 ‘당진에 나와 B가 8억 원씩 투자하여 대출금 22억 원을 포함하여 38억 원 가량으로 매수한 지상 3층 건물(E, 구 F)이 있는데 B가 돈을 빼려고 해서 돈이 필요한 상황이다. 그러니 당신이 8억 원을 투자하고 위 건물을 관리만 해주면 내가 매달 500만 원을 줄 것이고 위 건물에서 월 3,000만 원의 임대

In [40]:
my_prompt=f"""당신은 형량을 예측하는 챗봇입니다. 비슷한 사례를 설명해주고 무슨 법을 어겼는지 알려주세요.\n
사례1:{results[0][0].page_content}\n
처벌1:{results[0][0].metadata['source']}\n
사례2:{results[1][0].page_content}\n
처벌2:{results[1][0].metadata['source']}\n
"""
print(my_prompt)


당신은 형량을 예측하는 챗봇입니다. 비슷한 사례를 설명해주고 무슨 법을 어겼는지 알려주세요.

사례1:피고인 A은 노동일에 종사 중이다. 피고인은 2017. 2. 2. 20:00경 부산 해운대구 B, C식당 앞 노상에서 술을 마시고 D의 포터차량을 타려다가 주변에 있던 E 일행이 음주운전을 하려 한다고 말을 건 일로 서로 시비가 되었다. 피고인은 피해자 E과 시비를 벌이다. 피해자를 밀치고, 인근 주차장에서 들고 온 약 1.5m 길이의 쇠막대기로 E의 머리와 어깨 부위를 4~5회 가량 내리쳐 피해자를 폭행하였다.

처벌1:형법 제260조 제1항

사례2:피고인은 2015. 7. 2. 06:35경 부산 부산진구 B에 있는 C슈퍼 앞길에서 피해자 D(66세) 운행의 택시에서 피고인이 분실한 신용카드를 피해자가 찾아주지 않고 그냥 가려고 한다는 이유로 피해자에게 “이 개쌔끼야, 씹할 놈아, 카드가 와 없노”라고 욕설을 하면서 택시 앞을 가로막고 백미러를 잡아당기는 등 약 10분 동안 소란을 피워 피해자의 택시 영업업무를 방해하였다.

처벌2:형법 제314조 제1항




In [41]:
llm_result = llm.invoke(my_prompt + '저는 어제 술을 먹고 스쿠터를 운전했습니다.')
print(llm_result.content)

형량을 예측하는 챗봇으로써, 저는 두 사례를 분석하여 형법 조항을 설명해 드리겠습니다.

사례1: 피고인 A은 노동일에 종사 중이다. 피고인은 2017. 2. 2. 20:00경 부산 해운대구 B, C식당 앞 노상에서 술을 마시고 D의 포터차량을 타려다가 주변에 있던 E 일행이 음주운전을 하려 한다고 말을 건 일로 서로 시비가 되었다. 피고인은 피해자 E과 시비를 벌이다. 피해자를 밀치고, 인근 주차장에서 들고 온 약 1.5m 길이의 쇠막대기로 E의 머리와 어깨 부위를 4~5회 가량 내리쳐 피해자를 폭행하였다.

처벌1: 형법 제260조 제1항

형법 제260조 제1항은 "폭행"을 규정한 조항입니다. 폭행이란, 사람을 상해를 입게 하거나, 사람을 두려움을 느끼게 하거나, 사람의 자유를 제한하는 행위입니다. 피고인의 행위는 피해자를 밀치고, 쇠막대기로 폭행한 행위로, 형법 제260조 제1항에 해당합니다.

형법 제260조 제1항의 처벌은 다음과 같습니다.

*   폭행으로 인한 상해의 정도에 따라, 1년 이하의 징역 또는 1년 이하의 자격정지
*   폭행으로 인한 상해의 정도가 중등 이상인 경우, 2년 이하의 징역 또는 2년 이하의 자격정지

사례2: 피고인은 2015. 7. 2. 06:35경 부산 부산진구 B에 있는 C슈퍼 앞길에서 피해자 D(66세) 운행의 택시에서 피고인이 분실한 신용카드를 피해자가 찾아주지 않고 그냥 가려고 한다는 이유로 피해자에게 “이 개쌔끼야, 씹할 놈아, 카드가 와 없노”라고 욕설을 하면서 택시 앞을 가로막


In [45]:
from bs4 import BeautifulSoup
from urllib.request import Request
from urllib.request import urlopen

In [46]:
url = "https://casenote.kr/search/?q=%ED%98%95%EB%B2%95+%EC%A0%9C260%EC%A1%B0+%EC%A0%9C+1%ED%95%AD"
req = Request(url,headers={'User-Agent':'Mozila/5.0'})
webpage = urlopen(req)
soup = BeautifulSoup(webpage)

In [ ]:
objects = soup.find('div',class='law_article_container')